In [47]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
import pandas as pd
import shutil
import os 
import joblib

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.covariance import EllipticEnvelope
from matplotlib import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, make_scorer
from tqdm import tqdm
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import r2_score, make_scorer
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier

In [37]:
dataDF = pd.read_csv("mixed_dataset_w_features_v1.csv")
print(dataDF.shape)
num_resnet_features = len([n for n in dataDF.columns if "R18" in n])
num_hog_features = len([n for n in dataDF.columns if "hog" in n])
num_lbp_features = len([n for n in dataDF.columns if "lbp" in n])
print(f"Num features: HOG ({num_hog_features}) | LBP ({num_lbp_features}) | Resnet18 ({num_resnet_features})")
dataDF.head()

(2147, 865)
Num features: HOG (324) | LBP (25) | Resnet18 (512)


,filename,set,is_augmented,label,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,...,R18_503,R18_504,R18_505,R18_506,R18_507,R18_508,R18_509,R18_510,R18_511,R18_512
0,22_png.rf.31843c8a3d74795b58f1e718b5eed556.jpg,test,0,1,0.238798,0.206600,0.238798,0.085922,0.094069,0.034114,...,0.913866,0.524153,2.012202,1.042136,1.226978,0.258365,0.838587,0.108025,0.944895,0.414115
1,pose07_1_2_3.jpg,test,0,1,0.105851,0.010030,0.033357,0.044995,0.074824,0.032313,...,0.109936,0.047731,1.073908,1.094541,5.629246,0.000000,0.070865,1.159087,0.586770,0.579860
2,pose07_1_3_2.jpg,test,0,1,0.063727,0.015057,0.020632,0.029595,0.046573,0.015193,...,0.204381,0.188164,0.494481,2.225845,5.937500,0.000000,0.040817,0.642517,0.456618,0.256816
3,pose07_1_3_3.jpg,test,0,1,0.096622,0.028979,0.048910,0.069154,0.103628,0.033462,...,0.136055,0.144617,0.952040,1.355797,4.590340,0.087289,0.089996,1.743022,0.648363,0.576937
4,pose07_2_2_2.jpg,test,0,1,0.047135,0.024314,0.024849,0.028498,0.039671,0.017994,...,0.074890,0.561100,0.283085,1.325580,3.964945,0.000000,0.225154,0.180300,0.003969,0.152804


## Data split

In [4]:
trainDF = dataDF[dataDF.set == "train"]
Ytrain = trainDF[["label"]]
Xtrain = trainDF.drop(columns=["filename", "set", "is_augmented", "label"])
validDF = dataDF[dataDF.set == "validation"]
Yval = validDF[["label"]]
Xval = validDF.drop(columns=["filename", "set", "is_augmented", "label"])
testDF = dataDF[dataDF.set == "test"]
Ytest = testDF[["label"]]
Xtest = testDF.drop(columns=["filename", "set", "is_augmented", "label"])
print(trainDF.shape, validDF.shape, testDF.shape)
Xtrain.shape, Ytrain.shape, Xval.shape, Yval.shape, Xtest.shape, Ytest.shape

(1533, 865) (462, 865) (152, 865)


((1533, 861), (1533, 1), (462, 861), (462, 1), (152, 861), (152, 1))

## Prepocessing dataset

### Check missing values

In [5]:
missing_per_column = dataDF.isnull().sum()
print("Total missing value count:\n", sum(missing_per_column))

Total missing value count:
 0


### Check range of features

In [6]:
dataDF.describe()

,is_augmented,label,hog_1,hog_2,hog_3,hog_4,hog_5,hog_6,hog_7,hog_8,...,R18_503,R18_504,R18_505,R18_506,R18_507,R18_508,R18_509,R18_510,R18_511,R18_512
count,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,...,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000,2147.000000
mean,0.394038,0.272939,0.160152,0.075648,0.108387,0.073658,0.166399,0.038362,0.032559,0.026210,...,0.302849,0.861067,0.746226,1.311672,2.489634,0.495940,0.708239,0.747933,0.381378,0.488235
std,0.488757,0.445574,0.114052,0.074631,0.093214,0.074398,0.122324,0.035457,0.029767,0.026941,...,0.362570,0.695494,0.572148,0.750822,1.619127,0.557137,0.622040,0.629187,0.403147,0.457777
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.058795,0.020134,0.035532,0.025287,0.052662,0.015721,0.013800,0.010548,...,0.058475,0.321432,0.295397,0.782633,0.851015,0.118147,0.234841,0.280629,0.073847,0.120517
50%,0.000000,0.000000,0.126920,0.046755,0.069498,0.044860,0.134131,0.028439,0.025260,0.018692,...,0.180027,0.656088,0.657694,1.208026,2.651322,0.300628,0.539808,0.596000,0.273506,0.347002
75%,1.000000,1.000000,0.262556,0.108637,0.168094,0.096017,0.275888,0.049422,0.041808,0.032519,...,0.418664,1.262759,1.056380,1.715769,3.753782,0.676610,0.998782,1.055147,0.539087,0.734269
max,1.000000,1.000000,0.523001,0.370885,0.448290,0.375767,0.496237,0.249804,0.262132,0.267452,...,3.212196,5.186174,3.659534,6.014655,6.658197,4.599405,3.456626,4.588000,2.971324,3.271028


### Extract unaugmented train & valid data (no augmented samples in test set)

In [7]:
Xtrain_no_augmented = Xtrain[trainDF["is_augmented"] == 0]
Xval_no_augmented = Xval[validDF["is_augmented"] == 0]
Xtest_no_augmented = Xtest[testDF["is_augmented"] == 0]

Ytrain_no_augmented = Ytrain[trainDF["is_augmented"] == 0]
Yval_no_augmented = Yval[validDF["is_augmented"] == 0]
Ytest_no_augmented = Ytest[testDF["is_augmented"] == 0]

Xtrain_no_augmented.shape, Ytrain_no_augmented.shape, Xval_no_augmented.shape, Yval_no_augmented.shape, Xtest_no_augmented.shape, Ytest_no_augmented.shape

((955, 861), (955, 1), (196, 861), (196, 1), (150, 861), (150, 1))

### Compress data with PCA

#### Normal data

In [8]:
scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

In [9]:
pca = PCA(n_components=100)  # Specify the number of components to keep

# Fit and transform the data
Xtrain_scaled_pca100 = pca.fit_transform(Xtrain_scaled)
Xval_scaled_pca100 = pca.transform(Xval_scaled)
Xtest_scaled_pca100 = pca.transform(Xtest_scaled)

Xtrain_scaled_pca100.shape, Xval_scaled_pca100.shape, Xtest_scaled_pca100.shape

((1533, 100), (462, 100), (152, 100))

#### Unaugmented data

In [10]:
scaler_no_augmented = StandardScaler()
scaler_no_augmented.fit(Xtrain_no_augmented)
Xtrain_no_augmented_scaled = scaler_no_augmented.transform(Xtrain_no_augmented)
Xval_no_augmented_scaled = scaler_no_augmented.transform(Xval_no_augmented)
Xtest_no_augmented_scaled = scaler_no_augmented.transform(Xtest_no_augmented)

In [11]:
pca_no_augmented = PCA(n_components=100)  # Specify the number of components to keep

# Fit and transform the data
Xtrain_no_augmented_scaled_pca100 = pca_no_augmented.fit_transform(Xtrain_no_augmented_scaled)
Xval_no_augmented_scaled_pca100 = pca_no_augmented.transform(Xval_no_augmented_scaled)
Xtest_no_augmented_scaled_pca100 = pca_no_augmented.transform(Xtest_no_augmented_scaled)

Xtrain_no_augmented_scaled_pca100.shape, Xval_no_augmented_scaled_pca100.shape, Xtest_no_augmented_scaled_pca100.shape

((955, 100), (196, 100), (150, 100))

## Choose the best preprocessing method

In [12]:
def validate_preprocessing_method(xtrain, xval, ytrain, yval):
    if type(xtrain) is pd.DataFrame:
        xtrain = xtrain.to_numpy()
    if type(xval) is pd.DataFrame:
        xval = xval.to_numpy()
        
    mlp_estimator = MLPClassifier(solver="sgd", random_state=42)
    mlp_estimator.fit(xtrain, ytrain['label'])
    ypreds = mlp_estimator.predict(xval)
    acc = accuracy_score(yval["label"], ypreds)
    start_time = time.time()
    for i in range(100):
        mlp_estimator.predict(np.reshape(xtrain[np.random.randint(0,100),:], (1,-1)))
    end_time = time.time()
    print(f"Inference time per sample: {(end_time-start_time)/100} seconds")
    print(f"Accuracy: {acc}")
    return acc

### Normal dataset

In [29]:
validate_preprocessing_method(Xtrain, Xval, Ytrain, Yval)

Inference time per sample: 0.00029176950454711914 seconds
Accuracy: 0.9393939393939394


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9393939393939394

### Normal dataset - HOG features only

In [14]:
hog_columns = [f"hog_{i+1}" for i in range(num_hog_features)]
validate_preprocessing_method(Xtrain[hog_columns], Xval[hog_columns], Ytrain, Yval)

Inference time per sample: 0.0011310482025146484 seconds
Accuracy: 0.8961038961038961


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8961038961038961

### Normal dataset - LBP features only

In [15]:
lbp_columns = [f"lbp_{i+1}" for i in range(num_lbp_features)]
validate_preprocessing_method(Xtrain[lbp_columns], Xval[lbp_columns], Ytrain, Yval)

Inference time per sample: 7.724046707153321e-05 seconds
Accuracy: 0.48268398268398266


0.48268398268398266

### Normal dataset - Resnet18 features only

In [16]:
resnet_columns = [f"R18_{i+1}" for i in range(num_resnet_features)]
validate_preprocessing_method(Xtrain[resnet_columns], Xval[resnet_columns], Ytrain, Yval

Inference time per sample: 0.0007962727546691894 seconds
Accuracy: 0.9523809523809523


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9523809523809523

### No augmented dataset

In [26]:
validate_preprocessing_method(Xtrain_no_augmented, Xval_no_augmented, Ytrain_no_augmented, Yval_no_augmented)

Inference time per sample: 0.000319979190826416 seconds
Accuracy: 0.9744897959183674


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9744897959183674

### Scaled dataset

In [19]:
validate_preprocessing_method(Xtrain_scaled, Xval_scaled, Ytrain, Yval)

Inference time per sample: 0.0005600452423095703 seconds
Accuracy: 0.9805194805194806


0.9805194805194806

### PCA dataset

In [20]:
validate_preprocessing_method(Xtrain_scaled_pca100, Xval_scaled_pca100, Ytrain, Yval)

Inference time per sample: 0.00046150922775268556 seconds
Accuracy: 0.9761904761904762


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9761904761904762

### No augmented - Scaled dataset 

In [27]:
validate_preprocessing_method(Xtrain_no_augmented_scaled, Xval_no_augmented_scaled, Ytrain_no_augmented, Yval_no_augmented)

Inference time per sample: 0.0003251051902770996 seconds
Accuracy: 0.9642857142857143


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9642857142857143

### No augmented - PCA dataset

In [28]:
validate_preprocessing_method(Xtrain_no_augmented_scaled_pca100, Xval_no_augmented_scaled_pca100, Ytrain_no_augmented, Yval_no_augmented)

Inference time per sample: 0.00023311376571655273 seconds
Accuracy: 0.9540816326530612


C:\Users\Lenovo\miniconda3\envs\DA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.9540816326530612

## Choose suitable model family

In [33]:
Xtrain_scaled.shape, Xval_scaled.shape, Ytrain.shape, Yval.shape

((1533, 861), (462, 861), (1533, 1), (462, 1))

In [34]:
def automatic_scoring(x, y, estimator):
    average_score = cross_val_score(estimator=estimator, X=x, y=y, cv=5, scoring='accuracy', error_score="raise").mean()
    return average_score

In [40]:
rf_estimator = RandomForestClassifier()
s = automatic_scoring(Xtrain_scaled, Ytrain["label"], rf_estimator)
print(f"Score of Random Forest Regression: {s}")

knn_estimator = KNeighborsClassifier()
s = automatic_scoring(Xtrain_scaled, Ytrain["label"], knn_estimator)
print(f"Score of KNN: {s}")

mlp_estimator = MLPClassifier()
s = automatic_scoring(Xtrain_scaled, Ytrain["label"], mlp_estimator)
print(f"Score of MLP: {s}")

Score of Random Forest Regression: 0.9543846202976305
Score of KNN: 0.9315577696876796
Score of MLP: 0.9550403440420684


## Find an optimal parameters for the best model family

In [41]:
## do grid search to find the best model parameters
# we will choose the param around the default values to check if the direction would be better (increasing or decreasing compared to the default ones) 
params_hidden_layer_sizes = [25, 35, 50, 75, 100, 200, 500]
params_learning_rates = ['constant', 'invscaling', 'adaptive']
params_learning_rate_inits = [0.0001, 0.001, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.1]
params_max_iters = [50, 100, 200, 250, 300]
num_models = len(params_hidden_layer_sizes) * len(params_learning_rates) * len(params_learning_rate_inits) * len(params_max_iters)
print(f"Number of possible models: {num_models}")

Number of possible models: 840


In [43]:
## Train/val scheme
import warnings
warnings.filterwarnings("ignore")

best_params = None
best_acc = float('-inf')  # Initialize with negative infinity
start_time = time.time()
with tqdm(total=num_models) as pbar:
    for hls in params_hidden_layer_sizes:
        for lr in params_learning_rates:
            for lri in params_learning_rate_inits:
                for miter in params_max_iters:
                    model = MLPClassifier(solver="sgd", 
                                         hidden_layer_sizes=hls, 
                                         learning_rate=lr, 
                                         learning_rate_init=lri, 
                                         max_iter=miter,
                                         early_stopping=True,
                                         n_iter_no_change=4,
                                         random_state=42)
                    model.fit(Xtrain_scaled, Ytrain['label'])
                    ypreds = model.predict(Xval_scaled)
                    acc = accuracy_score(Yval["label"], ypreds)
                    
                    # Check if this set of hyperparameters is the best so far
                    if acc > best_acc:
                        best_acc = acc
                        best_params = {
                            'hidden_layer_sizes': hls,
                            'learning_rate': lr,
                            'learning_rate_init': lri,
                            'max_iter': miter
                        }
                    pbar.update(1)

end_time = time.time()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 840/840 [11:51<00:00,  1.18it/s]


In [45]:
# Print the best hyperparameters and R-squared
print(f'Best Parameters: {best_params}')
print(f'Best accuracy: {best_acc:.4f}')
print(f"Total searching time: {round(end_time - start_time,4)} seconds")

Best Parameters: {'hidden_layer_sizes': 75, 'learning_rate': 'constant', 'learning_rate_init': 0.05, 'max_iter': 50}
Best accuracy: 0.9848
Total searching time: 711.7765 seconds


## Validate on test set

In [46]:
model = MLPClassifier(solver="sgd", 
                     hidden_layer_sizes=best_params['hidden_layer_sizes'], 
                     learning_rate=best_params['learning_rate'], 
                     learning_rate_init=best_params['learning_rate_init'], 
                     max_iter=best_params['max_iter'],
                     early_stopping=True,
                     n_iter_no_change=4,
                     random_state=42)
model.fit(Xtrain_scaled, Ytrain['label'])
ytrainpreds = model.predict(Xtrain_scaled)
yvalpreds = model.predict(Xval_scaled)
ypreds = model.predict(Xtest_scaled)
acctrain = accuracy_score(Ytrain["label"], ytrainpreds)
accval = accuracy_score(Yval["label"], yvalpreds)
acc = accuracy_score(Ytest["label"], ypreds)
print(f'Train/val/test scoring comparison in case using only the training part:')
print(f'Accuracy training score: {acctrain:.2f}')
print(f'Accuracy validating score: {accval:.2f}')
print(f'Accuracy testing score: {acc:.2f}')

Train/val/test scoring comparison in case using only the training part:
Accuracy training score: 1.00
Accuracy validating score: 0.98
Accuracy testing score: 0.99


## Export the best model and some related material

In [49]:
joblib.dump(model, 'mlp_classifier_PblHogResnet18_861Features_model_v02.pkl')

['mlp_classifier_PblHogResnet18_861Features_model_v02.pkl']

In [50]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']